In [27]:
#!/usr/bin/env python
# coding: utf-8
'''
construct model for points cloud input based on XG-Boost
'''

import sys
root_dir = '../../'
sys.path.append(root_dir)
sys.path.append(root_dir + 'Data/corner/scripts')

import os
import psutil
import matplotlib
# matplotlib.use('agg') # so that plt works in command line
import numpy as np
import matplotlib.pyplot as plt
import sklearn.ensemble as sken
import sklearn.feature_selection as skfs
import Data_Feeder as feeder
import Metric_Recorder as recorder
import Model_Constructer as constructer

from optparse import OptionParser

In [21]:
data_path = root_dir + 'Data/corner/dw_19991231_162610_0.000000_0.000000/'
class_num = 2
line_re = '\t (?!3).*' # only corner radar
select_cols = [0, 1, 2,3,4,5,6,7,8,9,10] # all valid cols

dataset = feeder.Corner_Radar_Points_Gen_Feeder(data_path, 
                                                class_num=class_num, 
                                                use_onehot=False, 
                                                line_re=line_re, 
                                                select_cols=select_cols)
points, labels = dataset.get_all_data(allowed=True)
points = np.vstack(points)
labels = np.concatenate(labels)

print(dataset.feature_names)

['group_id', 'target_id', 'x', 'y', 'angle', 'distance', 'speed', 'rcs', 'v_x', 'v_y', 'lat_v']


In [24]:
std_points = points.copy()
std_points -= std_points.mean(axis=0)
std_points /= std_points.std(axis=0)
std_points[np.where(std_points != std_points)] = 0

minmax_point = std_points.copy()
minmax_point = (minmax_point - minmax_point.min(axis=0)) / (minmax_point.max(axis=0) - minmax_point.min(axis=0))
minmax_point[np.where(minmax_point != minmax_point)] = 0

/home/tangliyao/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tangliyao/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


## uni-variate selection

In [25]:
transformer = skfs.GenericUnivariateSelect(skfs.chi2, 'k_best')
transformer.fit(minmax_point, labels)
for n, s, p in zip(dataset.feature_names, transformer.scores_, transformer.pvalues_):
    print("%-10s %-10.5f %-10.5f" % (n,s,p))

group_id   25.34446   0.00000   
target_id  2158.44373 0.00000   
x          9.98067    0.00158   
y          8.70843    0.00317   
angle      705.98211  0.00000   
distance   2523.04772 0.00000   
speed      1041.65435 0.00000   
rcs        2271.05261 0.00000   
v_x        46.23807   0.00000   
v_y        0.23812    0.62556   
lat_v      nan        nan       


## recursive feature elimination

In [ ]:
rand_forest = sken.RandomForestClassifier(100)
rfe = skfs.RFE(estimator=rand_forest, n_features_to_select=1, step=1)
rfe.fit(points, labels)

In [ ]:
rfe.ranking_